In [16]:
import numpy as np
import cv2 as cv
import glob

In [17]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
rows = 7
cols = 7
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,6,0)
objp = np.zeros((rows*cols,3), np.float32)
objp[:,:2] = np.mgrid[0:cols,0:rows].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('./Images/*.png')

for fname in images:
    img = cv.imread(fname)
    print(fname)
    print(img.shape)

    h,w = img.shape[0], img.shape[1]
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    _, gray = cv.threshold(gray, 127, 255, cv.THRESH_BINARY)
    cv.imwrite(fname.replace(".png", "_gray.png").replace("Images", "Grayscale"), gray)

    # Find the chessboard corners
    print(cv.checkChessboard(gray, (cols, rows)))

    ret, corners = cv.findChessboardCornersSB(gray, (cols, rows), flags=0)
    print(ret) # chessboard detected or not

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (cols, rows), corners2, ret)
        cv.imwrite(fname.replace(".png", "_new.png").replace("Images", "Outputs"), img)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

./Images/3.png
(577, 433, 3)
True
True
./Images/5.png
(577, 433, 3)
True
True
./Images/2.png
(577, 433, 3)
True
True
./Images/1.png
(577, 433, 3)
True
True
./Images/4.png
(577, 433, 3)
True
True
./Images/6.png
(577, 433, 3)
True
True


In [18]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

rotation_matrices = []

# Iterate through each rotation vector and convert to a rotation matrix
for rvec in rvecs:
    rotation_matrix, _ = cv.Rodrigues(rvec)
    rotation_matrices.append(rotation_matrix)

In [19]:
print(ret)
print("-------------------------------------------")
print(mtx)
print("-------------------------------------------")
print(dist)
print("-------------------------------------------")
print(rotation_matrices)
print("-------------------------------------------")
print(tvecs)
print("-------------------------------------------")

print(newcameramtx)
print("-------------------------------------------")
print(roi)
print("-------------------------------------------")

0.9151344822909565
-------------------------------------------
[[440.44093701   0.         228.23975227]
 [  0.         439.21581056 261.18189485]
 [  0.           0.           1.        ]]
-------------------------------------------
[[ 9.96413382e-03  5.85713600e-01 -5.64285416e-03 -8.65693134e-04
  -1.30946286e+00]]
-------------------------------------------
[array([[ 0.99976092,  0.01073925,  0.01904676],
       [-0.01071719,  0.99994178, -0.00125966],
       [-0.01905918,  0.00105523,  0.9998178 ]]), array([[ 0.74952068, -0.50754825, -0.4249865 ],
       [ 0.62856848,  0.74704491,  0.21639216],
       [ 0.20765454, -0.42932352,  0.87895444]]), array([[ 0.9989129 ,  0.00872521, -0.04579181],
       [ 0.02209218,  0.77639546,  0.62985873],
       [ 0.0410482 , -0.63018565,  0.77535868]]), array([[ 0.99965728,  0.02157366,  0.01482905],
       [-0.02188205,  0.99954085,  0.02095862],
       [-0.01437009, -0.02127593,  0.99967036]]), array([[ 0.81490258,  0.57953544,  0.00851212],
   